In [6]:
# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
import time

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 引入 hashlib 模組
import hashlib

# 引入 regular expression 工具
import re

import random


options = webdriver.ChromeOptions()
options.add_argument("--headless")             
options.add_argument("--start-maximized")         
options.add_argument("--incognito")               
options.add_argument("--disable-popup-blocking ")

driver = webdriver.Chrome(options = options)

listStores =[]
def getStoresWeb():
    driver.get("https://www.ibon.com.tw/retail_inquiry.aspx#gsc.tab=0")
    time.sleep(2)

def getStores():
    for p in driver.find_elements(By.CSS_SELECTOR, 'div[style=" min-height:570px;"] div p'):
        p.click()
        time.sleep(2)
        
        storesAdd = driver.find_elements(By.CSS_SELECTOR,'table[class="font16"] tbody tr td:nth-child(3)')
       
        for add in storesAdd:
            addr = add.get_attribute('innerText')
            if addr != "地址":
                listStores.append({
                    'address':addr
                })
        
        time.sleep(2)
        
def saveJson():
    fp = open("./cvs_stores.json", "w", encoding="utf-8")
    fp.write( json.dumps(listStores, ensure_ascii=False) )
    fp.close()
    
def readJson():
    fp = open("./cvs_stores.json","r",encoding="utf-8")
    strJson = fp.read()
    fp.close()

def get_coordinate(addr):    
    driver.get("http://www.map.com.tw/")
    search = driver.find_element_by_id("searchWord")
    search.clear()
    search.send_keys(addr)
    driver.find_element_by_xpath("/html/body/form/div[10]/div[2]/img[2]").click() 
    time.sleep(random.randint(30,90))
    try:
     
        iframe = driver.find_elements_by_tag_name("iframe")[1]
        iframe = driver.find_elements_by_class_name("winfoIframe")[0]
        driver.switch_to.frame(iframe)
        coor_btn = driver.find_element_by_xpath("/html/body/form/div[4]/table/tbody/tr[3]/td/table/tbody/tr/td[2]")
        coor_btn.click()
        coor = driver.find_element_by_xpath("/html/body/form/div[5]/table/tbody/tr[2]/td")
        coor = coor.text.strip().split(" ")
        lat = coor[-1].split("：")[-1]
        log = coor[0].split("：")[-1]
       
    except IndexError:
        lat = 0
        log = 0
    except NameError:
        print("被抓到了")
        fp = open(f"./exception_cvs_stores_location.json", "w", encoding="utf-8")
        fp.write( json.dumps(listAddr, ensure_ascii=False) )
        fp.close()
        
    except TypeError:      
        time.sleep(30)
        
    
    return (lat, log)

def getLocation():
    fp = open("./cvs_stores.json","r",encoding="utf-8")
    strJson = fp.read()
    fp.close()
    driver.get("http://www.map.com.tw")
    listAddr = json.loads(strJson)
    print("-"*10+"下載中,請勿關閉視窗"+"-"*10)
    for index,obj in enumerate(listAddr): 
        
        
        listAddr[index]['location'] = get_coordinate(listAddr[index]['address'])
        time.sleep(random.randint(4,10))
        
        if ((index+1) % 300 == 0):
            
            fp = open(f"./{index}_cvs_stores_location.json", "w", encoding="utf-8")
            fp.write( json.dumps(listAddr, ensure_ascii=False) )
            fp.close()
            print("每到300倍數等待十分鐘")
            time.sleep(random.randint(600,660))
            
    fp = open("./cvs_stores_location.json", "w", encoding="utf-8")
    fp.write( json.dumps(listAddr, ensure_ascii=False) )
    fp.close()

    
def quit():
    driver.quit()
     
if __name__ == "__main__":
    startTime = time.time()
#    getStoresWeb()
#    getStores()
#    saveJson()
#    readJson()
    getLocation()
    quit()
    print(f"執行總花費時間: {time.time() - startTime}")

0
執行總花費時間: 31.696043014526367
